## <span style="color:darkgreen">IBM Watson AutoAI - Generated Scikit-learn Notebook v1.14.4</span>

Consider these tips for working with an auto-generated notebook:
- Notebook code generated using AutoAI will execute successfully. If you modify the notebook, we cannot guarantee it will run successfully.
- This pipeline is optimized for the original data set. The pipeline might fail or produce sub-optimum results if used with different data.  If you want to use a different data set, consider retraining the AutoAI experiment to generate a new pipeline. For more information, see <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/autoai-notebook.html">Cloud Platform</a> 
- Before modifying the pipeline or trying to re-fit the pipeline, consider that the code converts dataframes to numpy arrays before fitting the pipeline (a current restriction of the preprocessor pipeline).


<a id="content"></a>
## Notebook content

This notebook contains a Scikit-learn representation of AutoAI pipeline. This notebook introduces commands for getting data, training the model, and testing the model. 

Some familiarity with Python is helpful. This notebook uses Python 3.7 and scikit-learn 0.23.1.

## Notebook goals

-  Scikit-learn pipeline definition
-  Pipeline training 
-  Pipeline evaluation

## Contents

This notebook contains the following parts:

**[Setup](#setup)**      
&nbsp;&nbsp;[Package installation](#install)   
&nbsp;&nbsp;[AutoAI experiment metadata](#variables_definition)   
**[Pipeline inspection](#inspection)**      
&nbsp;&nbsp;[Read training data](#read)    
&nbsp;&nbsp;[Train and test data split](#split)    
&nbsp;&nbsp;[Make pipeline](#preview_model_to_python_code)      
&nbsp;&nbsp;[Train pipeline model](#train)  
&nbsp;&nbsp;[Test pipeline model](#test_model)  
**[Next steps](#next_steps)**   
**[Copyrights](#copyrights)**

<a id="setup"></a>
# Setup

<a id="install"></a>
## Package installation
Before you use the sample code in this notebook, install the following packages:
 - ibm_watson_machine_learning,
 - autoai-libs.


In [1]:
!pip install ibm-watson-machine-learning | tail -n 1
!pip install -U autoai-libs | tail -n 1

<a id="variables_definition"></a>
## AutoAI experiment metadata
The following cell contains the training data connection details.  
**Note**: The connection might contain authorization credentials, so be careful when sharing the notebook.

In [2]:
# @hidden_cell
from ibm_watson_machine_learning.helpers import DataConnection, S3Connection, S3Location

training_data_reference = [DataConnection(
    connection=S3Connection(
        api_key='ZLWT8yBNpBtpd3VBQ6BOLVOUOZNNrv9doE1HBb7e_utI',
        auth_endpoint='https://iam.bluemix.net/oidc/token/',
        endpoint_url='https://s3-api.us-geo.objectstorage.softlayer.net'
    ),
        location=S3Location(
        bucket='paymentplanprediction-donotdelete-pr-t89pyddzg6ashw',
        path='historical_payments_customer_detail_balanced.csv'
    )),
]
training_result_reference = DataConnection(
    connection=S3Connection(
        api_key='ZLWT8yBNpBtpd3VBQ6BOLVOUOZNNrv9doE1HBb7e_utI',
        auth_endpoint='https://iam.bluemix.net/oidc/token/',
        endpoint_url='https://s3-api.us-geo.objectstorage.softlayer.net'
    ),
    location=S3Location(
        bucket='paymentplanprediction-donotdelete-pr-t89pyddzg6ashw',
        path='auto_ml/9efe943b-1e50-4294-9bb2-fec5b02d39d4/wml_data/9cb7e736-4ffb-4240-831c-7040d2ed675c/data/automl',
        model_location='auto_ml/9efe943b-1e50-4294-9bb2-fec5b02d39d4/wml_data/9cb7e736-4ffb-4240-831c-7040d2ed675c/data/automl/cognito_output/Pipeline5/model.pickle',
        training_status='auto_ml/9efe943b-1e50-4294-9bb2-fec5b02d39d4/wml_data/9cb7e736-4ffb-4240-831c-7040d2ed675c/training-status.json'
    ))

Following cell contains input parameters provided to run the AutoAI experiment in Watson Studio.

In [3]:
experiment_metadata = dict(
   prediction_type='classification',
   prediction_column='MISSED_PAYMENT',
   test_size=0.1,
   scoring='accuracy',
   project_id='3a43d8dd-4a82-444b-af95-97b8dee5c72e',
   space_id='None',
   deployment_url='https://us-south.ml.cloud.ibm.com',
   csv_separator=',',
   random_state=33,
   max_number_of_estimators=2,
   daub_include_only_estimators=None,
   training_data_reference=training_data_reference,
   training_result_reference=training_result_reference,
   positive_label='Missed Payment'
)

<a id="inspection"></a>
# Pipeline inspection

<a id="read"></a>
## Read training data

Retrieve training dataset from AutoAI experiment as pandas DataFrame.

In [4]:
df = training_data_reference[0].read(csv_separator=experiment_metadata['csv_separator'])
df.dropna('rows', how='any', subset=[experiment_metadata['prediction_column']], inplace=True)

<a id="split"></a>
##  Train and test data split

In [5]:
from sklearn.model_selection import train_test_split

df.drop_duplicates(inplace=True)
X = df.drop([experiment_metadata['prediction_column']], axis=1).values
y = df[experiment_metadata['prediction_column']].values

train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=experiment_metadata['test_size'],
                                                    stratify=y, random_state=experiment_metadata['random_state'])

<a id="preview_model_to_python_code"></a>
## Make pipeline
In the next cell, you can find the Scikit-learn definition of the selected AutoAI pipeline.

Import statements.

In [6]:
from autoai_libs.transformers.exportable import ColumnSelector
from autoai_libs.transformers.exportable import NumpyColumnSelector
from autoai_libs.transformers.exportable import CompressStrings
from autoai_libs.transformers.exportable import NumpyReplaceMissingValues
from autoai_libs.transformers.exportable import NumpyReplaceUnknownValues
from autoai_libs.transformers.exportable import boolean2float
from autoai_libs.transformers.exportable import CatImputer
from autoai_libs.transformers.exportable import CatEncoder
import numpy as np
from autoai_libs.transformers.exportable import float32_transform
from sklearn.pipeline import make_pipeline
from autoai_libs.transformers.exportable import FloatStr2Float
from autoai_libs.transformers.exportable import NumImputer
from autoai_libs.transformers.exportable import OptStandardScaler
from sklearn.pipeline import make_union
from autoai_libs.transformers.exportable import NumpyPermuteArray
from autoai_libs.cognito.transforms.transform_utils import TA1
import autoai_libs.utils.fc_methods
from autoai_libs.cognito.transforms.transform_utils import FS1
from sklearn.tree import DecisionTreeClassifier

#### Pre-processing & Estimator.

In [7]:
column_selector = ColumnSelector(
    columns_indices_list=[
        1,
        2,
        3,
        4,
        5,
        6,
        7,
        8,
        9,
        10,
        11,
        16,
        17,
        18,
        19,
        20,
        21,
        22,
        23,
        24,
        25,
        26,
        27,
        28,
        29,
        30,
        31,
        32,
        33,
        34,
        35,
        36,
        37,
        38,
        40,
        45,
        46,
    ]
)
numpy_column_selector_0 = NumpyColumnSelector(
    columns=[
        2,
        11,
        12,
        13,
        14,
        15,
        16,
        17,
        18,
        19,
        20,
        21,
        22,
        23,
        24,
        25,
        26,
        27,
        28,
        29,
        30,
        31,
        32,
        35,
        36,
    ]
)
compress_strings = CompressStrings(
    compress_type="hash",
    dtypes_list=[
        "char_str",
        "int_num",
        "int_num",
        "int_num",
        "int_num",
        "int_num",
        "int_num",
        "int_num",
        "int_num",
        "char_str",
        "int_num",
        "int_num",
        "int_num",
        "int_num",
        "int_num",
        "int_num",
        "char_str",
        "char_str",
        "char_str",
        "char_str",
        "char_str",
        "char_str",
        "int_num",
        "int_num",
        "char_str",
    ],
    missing_values_reference_list=["", "-", "?", float("nan")],
    misslist_list=[
        [],
        [],
        [],
        [],
        [],
        [],
        [],
        [],
        [],
        [],
        [],
        [],
        [],
        [],
        [],
        [],
        [],
        [],
        [],
        [],
        [],
        [],
        [],
        [],
        [],
    ],
)
numpy_replace_missing_values_0 = NumpyReplaceMissingValues(
    missing_values=[], filling_values=float("nan")
)
numpy_replace_unknown_values = NumpyReplaceUnknownValues(
    filling_values=float("nan"),
    filling_values_list=[
        float("nan"),
        float("nan"),
        float("nan"),
        float("nan"),
        float("nan"),
        float("nan"),
        float("nan"),
        float("nan"),
        float("nan"),
        float("nan"),
        float("nan"),
        float("nan"),
        float("nan"),
        float("nan"),
        float("nan"),
        float("nan"),
        float("nan"),
        float("nan"),
        float("nan"),
        float("nan"),
        float("nan"),
        float("nan"),
        float("nan"),
        float("nan"),
        float("nan"),
    ],
    known_values_list=[
        [
            169836834567204038179966570894283554345,
            322803543197566830139700178970154925496,
            209181221264684009591229952740346738033,
            17788651139535530962259434531561804375,
            327176509795513342126021012545780561845,
            77420392375083351592742626634322842842,
        ],
        [
            15,
            16,
            17,
            18,
            19,
            20,
            21,
            22,
            23,
            24,
            25,
            26,
            27,
            28,
            29,
            30,
            31,
            32,
            33,
            34,
            35,
            36,
            37,
            38,
            39,
            40,
            41,
            42,
            43,
            44,
            45,
            46,
            47,
            48,
            49,
            50,
            51,
            52,
            53,
            54,
            55,
            56,
            57,
            58,
            59,
            60,
            61,
            63,
            64,
            65,
            66,
            68,
            69,
            70,
            71,
            72,
            74,
            75,
            76,
            77,
            78,
            79,
        ],
        [0, 1],
        [0, 1],
        [0, 1],
        [0, 1],
        [0, 1],
        [0, 1],
        [0, 1],
        [
            339914797681135161069053613952454376331,
            310682768561293208144752856853279300097,
            67173585239538071839587539288433045831,
        ],
        [0, 1],
        [0, 1],
        [0, 1],
        [0, 1],
        [0, 1],
        [0, 1],
        [
            307233733497834568525459704737014618558,
            121370508098484644977732955873471414694,
            6543286095828419275771321000144814902,
            48876320958914818052114633595448615891,
            110469894902109575489983869353176417187,
        ],
        [
            140114708448418632577632402066430035116,
            124597455836222876990001734982037886062,
            101526347069924884539065843501838193131,
        ],
        [
            326253301125662860281687750756690785583,
            273210353247234602564054441676862833744,
            302290990792076731556337512361380154350,
            143628117342068315368254630461460844700,
            249995396699920924073666887213313108912,
            243201052615821965146514263103464324903,
            119538630647902979951616946415647563536,
            261657353565822963585293184140899076613,
            99651907288817954462213660049441930213,
        ],
        [
            276215275525073243129443018166533317850,
            314453448426715438145956090661358255141,
            95319007259848359155306824649621382536,
            61125751469413322076599764069662232065,
        ],
        [
            315380121665422614716550309578519414425,
            240512810358043060361441826343089376772,
            197239169984543494630885295882584127830,
            20444808171318292122905678980475303515,
            228859325915791505068217040611500381639,
            198303222032484114342112753715176650929,
            277858879624756597381704479412853426130,
            209700651107874524918013128490797585337,
            326661061374373185892543585874572058679,
            180900830972951739246963077271060433534,
        ],
        [
            153486499799404094467727568916354293838,
            156971208605157160966856555059464064936,
            128080626020449029580470050252318967524,
            203128837411296815074725089422714954118,
            245974085598099695416043785224837302844,
            23776053449641394854451841780039140909,
            230267001313728654787380132776226734787,
        ],
        [
            201807,
            201808,
            201809,
            201810,
            201811,
            201812,
            201901,
            201902,
            201903,
            201904,
            201905,
            201906,
        ],
        [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
        [
            85656995390257636701648734804000448873,
            132720696610303284419807397392052094621,
        ],
    ],
    missing_values_reference_list=["", "-", "?", float("nan")],
)
cat_imputer = CatImputer(
    strategy="most_frequent",
    missing_values=float("nan"),
    sklearn_version_family="23",
)
cat_encoder = CatEncoder(
    encoding="ordinal",
    categories="auto",
    dtype=np.float64,
    handle_unknown="error",
    sklearn_version_family="23",
)
pipeline_0 = make_pipeline(
    numpy_column_selector_0,
    compress_strings,
    numpy_replace_missing_values_0,
    numpy_replace_unknown_values,
    boolean2float(),
    cat_imputer,
    cat_encoder,
    float32_transform(),
)
numpy_column_selector_1 = NumpyColumnSelector(
    columns=[0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 33, 34]
)
float_str2_float = FloatStr2Float(
    dtypes_list=[
        "float_num",
        "float_num",
        "float_num",
        "float_num",
        "float_num",
        "float_num",
        "float_num",
        "float_num",
        "float_num",
        "float_num",
        "float_num",
        "float_num",
    ],
    missing_values_reference_list=[],
)
numpy_replace_missing_values_1 = NumpyReplaceMissingValues(
    missing_values=[], filling_values=float("nan")
)
num_imputer = NumImputer(strategy="median", missing_values=float("nan"))
opt_standard_scaler = OptStandardScaler(
    num_scaler_copy=None,
    num_scaler_with_mean=None,
    num_scaler_with_std=None,
    use_scaler_flag=False,
)
pipeline_1 = make_pipeline(
    numpy_column_selector_1,
    float_str2_float,
    numpy_replace_missing_values_1,
    num_imputer,
    opt_standard_scaler,
    float32_transform(),
)
union = make_union(pipeline_0, pipeline_1)
numpy_permute_array = NumpyPermuteArray(
    axis=0,
    permutation_indices=[
        2,
        11,
        12,
        13,
        14,
        15,
        16,
        17,
        18,
        19,
        20,
        21,
        22,
        23,
        24,
        25,
        26,
        27,
        28,
        29,
        30,
        31,
        32,
        35,
        36,
        0,
        1,
        3,
        4,
        5,
        6,
        7,
        8,
        9,
        10,
        33,
        34,
    ],
)
ta1_0 = TA1(
    fun=np.sqrt,
    name="sqrt",
    datatypes=["numeric"],
    feat_constraints=[
        autoai_libs.utils.fc_methods.is_non_negative,
        autoai_libs.utils.fc_methods.is_not_categorical,
    ],
    col_names=[
        "OVERDUE_BALANCE",
        "BASE_USAGE",
        "CREDIT_HISTORY",
        "ALTERNATE_USAGE",
        "STANDING_CHARGE",
        "BASE_CHARGE",
        "ALTERNATE_CHARGE",
        "LEVY",
        "TAX",
        "TOTAL_NET",
        "TOTAL_TO_PAY",
        "AGE",
        "IS_REGISTERED_FOR_ALERTS",
        "OWNS_HOME",
        "COMPLAINTS",
        "HAS_THERMOSTAT",
        "HAS_HOME_AUTOMATION",
        "PV_ZONING",
        "WIND_ZONING",
        "SMART_METER_COMMENTS",
        "IS_CAR_OWNER",
        "HAS_EV",
        "HAS_PV",
        "HAS_WIND",
        "EBILL",
        "IN_WARRANTY",
        "CITY",
        "MARITAL_STATUS",
        "EDUCATION",
        "SEGMENT",
        "EMPLOYMENT",
        "BUILDING_TYPE",
        "BILLING_MONTH",
        "RATIO_THIS_MONTH_BASE_USAGE_VS_LAST_MONTH",
        "RATIO_THIS_MONTH_ALTERNATE_USAGE_VS_LAST_MONTH",
        "BILLING_MONTH_NUMBER",
        "Partition",
    ],
    col_dtypes=[
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
    ],
)
fs1_0 = FS1(
    cols_ids_must_keep=range(0, 37),
    additional_col_count_to_keep=20,
    ptype="classification",
)
ta1_1 = TA1(
    fun=np.square,
    name="square",
    datatypes=["numeric"],
    feat_constraints=[autoai_libs.utils.fc_methods.is_not_categorical],
    col_names=[
        "OVERDUE_BALANCE",
        "BASE_USAGE",
        "CREDIT_HISTORY",
        "ALTERNATE_USAGE",
        "STANDING_CHARGE",
        "BASE_CHARGE",
        "ALTERNATE_CHARGE",
        "LEVY",
        "TAX",
        "TOTAL_NET",
        "TOTAL_TO_PAY",
        "AGE",
        "IS_REGISTERED_FOR_ALERTS",
        "OWNS_HOME",
        "COMPLAINTS",
        "HAS_THERMOSTAT",
        "HAS_HOME_AUTOMATION",
        "PV_ZONING",
        "WIND_ZONING",
        "SMART_METER_COMMENTS",
        "IS_CAR_OWNER",
        "HAS_EV",
        "HAS_PV",
        "HAS_WIND",
        "EBILL",
        "IN_WARRANTY",
        "CITY",
        "MARITAL_STATUS",
        "EDUCATION",
        "SEGMENT",
        "EMPLOYMENT",
        "BUILDING_TYPE",
        "BILLING_MONTH",
        "RATIO_THIS_MONTH_BASE_USAGE_VS_LAST_MONTH",
        "RATIO_THIS_MONTH_ALTERNATE_USAGE_VS_LAST_MONTH",
        "BILLING_MONTH_NUMBER",
        "Partition",
        "sqrt(OVERDUE_BALANCE)",
        "sqrt(BASE_USAGE)",
        "sqrt(ALTERNATE_USAGE)",
        "sqrt(BASE_CHARGE)",
        "sqrt(ALTERNATE_CHARGE)",
        "sqrt(LEVY)",
        "sqrt(TAX)",
        "sqrt(TOTAL_NET)",
        "sqrt(TOTAL_TO_PAY)",
        "sqrt(AGE)",
        "sqrt(RATIO_THIS_MONTH_BASE_USAGE_VS_LAST_MONTH)",
        "sqrt(RATIO_THIS_MONTH_ALTERNATE_USAGE_VS_LAST_MONTH)",
    ],
    col_dtypes=[
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
    ],
)
fs1_1 = FS1(
    cols_ids_must_keep=range(0, 37),
    additional_col_count_to_keep=20,
    ptype="classification",
)
decision_tree_classifier = DecisionTreeClassifier(
    class_weight="balanced",
    max_depth=4,
    max_features=0.9991605525579291,
    min_samples_leaf=5,
    min_samples_split=4,
    presort=False,
    random_state=33,
)


Pipeline.

In [8]:
pipeline = make_pipeline(
    column_selector,
    union,
    numpy_permute_array,
    ta1_0,
    fs1_0,
    ta1_1,
    fs1_1,
    decision_tree_classifier,
)

<a id="train"></a>
## Train pipeline model


### Define scorer from the optimization metric
This cell constructs the cell scorer based on the experiment metadata.

In [9]:
from sklearn.metrics import get_scorer

scorer = get_scorer(experiment_metadata['scoring'])

<a id="test_model"></a>
### Fit pipeline model
In this cell, the pipeline is fitted.

In [10]:
pipeline.fit(train_X,train_y)

Pipeline(steps=[('columnselector',
                 ColumnSelector(columns_indices_list=[1, 2, 3, 4, 5, 6, 7, 8, 9,
                                                      10, 11, 16, 17, 18, 19,
                                                      20, 21, 22, 23, 24, 25,
                                                      26, 27, 28, 29, 30, 31,
                                                      32, 33, 34, ...])),
                ('featureunion',
                 FeatureUnion(transformer_list=[('pipeline-1',
                                                 Pipeline(steps=[('numpycolumnselector',
                                                                  NumpyColumnSelector(columns=[2,
                                                                                               11,
                                                                                               12,
                                                                                               

<a id="test_model"></a>
## Test pipeline model

Score the fitted pipeline with the generated scorer using the holdout dataset.

In [11]:
score = scorer(pipeline, test_X, test_y)
print(score)

0.8714285714285714


<a id="next_steps"></a>
# Next steps

#### [Model deployment as webservice](https://github.com/IBM/watson-machine-learning-samples/tree/master/cloud/notebooks/python_sdk/deployments/autoai)
#### [Run AutoAI experiment with python SDK](https://github.com/IBM/watson-machine-learning-samples/tree/master/cloud/notebooks/python_sdk/experiments/autoai)  

<a id="copyrights"></a>
### Copyrights

Licensed Materials - Copyright © 2020 IBM. This notebook and its source code are released under the terms of the ILAN License.
Use, duplication disclosure restricted by GSA ADP Schedule Contract with IBM Corp.

**Note:** The auto-generated notebooks are subject to the International License Agreement for Non-Warranted Programs  
(or equivalent) and License Information document for Watson Studio Auto-generated Notebook (License Terms),  
such agreements located in the link below. Specifically, the Source Components and Sample Materials clause  
included in the License Information document for Watson Studio Auto-generated Notebook applies to the auto-generated notebooks.  

By downloading, copying, accessing, or otherwise using the materials, you agree to the <a href="http://www14.software.ibm.com/cgi-bin/weblap/lap.pl?li_formnum=L-AMCU-BHU2B7&title=IBM%20Watson%20Studio%20Auto-generated%20Notebook%20V2.1">License Terms</a>  

___